<a href="https://colab.research.google.com/github/chenqinkai/bert/blob/master/colab/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pprint
import os
import sys
import tensorflow as tf
import json

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.77.161.90:8470

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 16296186146782398233),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 7831519077706974132),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9290374637001085207),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 12651468990974235434),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14457625127729658402),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 8002924094293152988),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17

clone my git repo which contains code to run

In [0]:
!rm -rf bert_repo
!test -d bert_repo || git clone https://github.com/chenqinkai/bert.git bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

Cloning into 'bert_repo'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 385 (delta 0), reused 1 (delta 0), pack-reused 382
Receiving objects: 100% (385/385), 272.36 KiB | 3.63 MiB/s, done.
Resolving deltas: 100% (221/221), done.


try to get bert models already saved by google

In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


load files from my bucket and create output directory

In [0]:
EPOCH = 3 #@param {type:"integer"}
BATCH_SIZE = 32 #@param {type:"integer"}
LEARNING_RATE = 2e-5 #@param {type:"number"}
MAX_LEN = 96 #@param {type:"integer"}

INPUT_DIR = 'gs://bert-news-sentiment/reuters/horizon_3'
OUTPUT_DIR = 'gs://bert-news-sentiment/bert/models/reuters/horizon-3_percentile-10_epoch-%d_batch-%d_lr-%d_maxlen-%d' % (
    EPOCH, BATCH_SIZE, LEARNING_RATE * 1e5, MAX_LEN)
tf.gfile.MakeDirs(OUTPUT_DIR)
tf.gfile.MakeDirs(os.path.join(OUTPUT_DIR, "prediction"))
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://bert-news-sentiment/bert/models/reuters/horizon-3_percentile-10_epoch-3_batch-32_lr-2_maxlen-96 *****


In [0]:
TRAIN_TSV_NAME = "training_horizon_3_percentile_10.tsv" #@param {type:"string"}
DEV_TSV_NAME = "test_horizon_3.tsv" #@param {type:"string"}

!python bert_repo/run_classifier.py \
  --task_name=REUTERS \
  --do_train=true \
  --do_eval=true \
  --data_dir=$INPUT_DIR \
  --vocab_file=$BERT_PRETRAINED_DIR/vocab.txt \
  --bert_config_file=$BERT_PRETRAINED_DIR/bert_config.json \
  --init_checkpoint=$BERT_PRETRAINED_DIR/bert_model.ckpt \
  --max_seq_length=$MAX_LEN \
  --train_batch_size=$BATCH_SIZE \
  --learning_rate=$LEARNING_RATE \
  --num_train_epochs=$EPOCH \
  --output_dir=$OUTPUT_DIR \
  --use_tpu=true \
  --tpu_name=$TPU_ADDRESS \
  --train_tsv_name=$TRAIN_TSV_NAME \
  --dev_tsv_name=$DEV_TSV_NAME



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

0
INFO:tensorflow:Using config: {'_model_dir': 'gs://bert-news-sentiment/bert/models/reuters/horizon-3_percentile-10_epoch-3_batch-32_lr-2_maxlen-96', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.77.161.90:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f93db3b5358>

In [0]:
# MODEL_DIR = "gs://bert-news-sentiment/bert/models/reuters/horizon_3_percentile_10_epoch_10" #@param {type:"string"}
MODEL_DIR = OUTPUT_DIR
PREDICT_COL_NAME="return_3" #@param {type:"string"}
TEST_TSV_NAME = "test_horizon_3.tsv" #@param {type:"string"}

!python bert_repo/run_classifier.py \
  --task_name=REUTERS \
  --do_predict=true \
  --data_dir=$INPUT_DIR \
  --vocab_file=$BERT_PRETRAINED_DIR/vocab.txt \
  --bert_config_file=$BERT_PRETRAINED_DIR/bert_config.json \
  --init_checkpoint=$MODEL_DIR \
  --max_seq_length=$MAX_LEN \
  --output_dir=$MODEL_DIR/prediction \
  --use_tpu=True \
  --tpu_name=$TPU_ADDRESS \
  --test_tsv_name=$TEST_TSV_NAME \
  --predict_col_name=$PREDICT_COL_NAME


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': 'gs://bert-news-sentiment/bert/models/reuters/horizon-3_percentile-10_epoch-3_batch-32_lr-2_maxlen-96/prediction', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.77.161.90:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd6